In [ ]:
import csv
import os
from datetime import datetime
import matplotlib.pyplot as plt

DATA_FILE = 'expenses.csv'

In [ ]:
def initialize_file():
#  Ensure the CSV file exists with the correct header.
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['date', 'category', 'amount', 'description'])


In [ ]:
def load_expenses():
    expenses = []
    with open(DATA_FILE, 'r', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # Convert types
            row['amount'] = float(row['amount'])
            row['date'] = datetime.strptime(row['date'], '%Y-%m-%d').date()
            expenses.append(row)
    return expenses


In [ ]:
def save_expenses(expenses):
    
    with open(DATA_FILE, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['date', 'category', 'amount', 'description'])
        for exp in expenses:
            writer.writerow([
                exp['date'].isoformat(),
                exp['category'],
                exp['amount'],
                exp['description']
            ])


def add_expense():
#      Prompt user for expense details and append to CSV.

    date_str = input('Enter date (YYYY-MM-DD): ')
    category = input('Enter category: ')
    amount = float(input('Enter amount: '))
    description = input('Enter description: ')
    expenses = load_expenses()
    expenses.append({
        'date': datetime.strptime(date_str, '%Y-%m-%d').date(),
        'category': category,
        'amount': amount,
        'description': description
    })
    save_expenses(expenses)
    print('Expense added!')


def view_expenses():
    expenses = load_expenses()
    if not expenses:
        print('No expenses found.')
        return
    for i, exp in enumerate(expenses, start=1):
        print(f"{i}. {exp['date']} | {exp['category']} | ₹{exp['amount']} | {exp['description']}")


def delete_expense():
    expenses = load_expenses()
    view_expenses()
    idx = int(input('Enter record number to delete: ')) - 1
    if 0 <= idx < len(expenses):
        removed = expenses.pop(idx)
        save_expenses(expenses)
        print(f"Deleted: {removed}")
    else:
        print('Invalid record number.')


def update_expense():
    expenses = load_expenses()
    view_expenses()
    idx = int(input('Enter record number to update: ')) - 1
    if 0 <= idx < len(expenses):
        exp = expenses[idx]
        print('Press Enter to keep current value.')
        date_str = input(f"Date ({exp['date']}): ")
        if date_str:
            exp['date'] = datetime.strptime(date_str, '%Y-%m-%d').date()
        cat = input(f"Category ({exp['category']}): ")
        if cat:
            exp['category'] = cat
        amt = input(f"Amount ({exp['amount']}): ")
        if amt:
            exp['amount'] = float(amt)
        desc = input(f"Description ({exp['description']}): ")
        if desc:
            exp['description'] = desc
        save_expenses(expenses)
        print('Expense updated!')
    else:
        print('Invalid record number.')


In [ ]:
def plot_summary():
    """
    Generate two plots:
    1) Line chart of monthly totals.
    2) Pie chart of expenses by category.
    """
    expenses = load_expenses()
    if not expenses:
        print('No data to plot.')
        return

    # Monthly totals
    monthly = {}
    for exp in expenses:
        m = exp['date'].strftime('%Y-%m')
        monthly[m] = monthly.get(m, 0) + exp['amount']
    months = sorted(monthly)
    totals = [monthly[m] for m in months]

    plt.figure()
    plt.plot(months, totals, marker='o')
    plt.title('Monthly Expenses')
    plt.xlabel('Month')
    plt.ylabel('Amount (₹)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Category breakdown
    cat_totals = {}
    for exp in expenses:
        cat_totals[exp['category']] = cat_totals.get(exp['category'], 0) + exp['amount']

    plt.figure()
    plt.pie(
        list(cat_totals.values()),
        labels=list(cat_totals.keys()),
        autopct='%1.1f%%'
    )
    plt.title('Expenses by Category')
    plt.show()


In [ ]:
import csv
from collections import defaultdict
from datetime import datetime

FILENAME = "expenses.csv"

def monthly_category_summary():
    summary = defaultdict(lambda: defaultdict(float))

    with open(FILENAME, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Extract month from date
            date_obj = datetime.strptime(row['Date'], "%Y-%m-%d %H:%M:%S")
            month = date_obj.strftime("%B %Y") 

            category = row['Category']
            amount = float(row['Amount'])
            
            summary[month][category] += amount


    for month, categories in summary.items():
        print(f"\n {month}")
        for category, total in categories.items():
            print(f"   {category}: ₹{total}")


In [ ]:
def main():
    initialize_file()
    while True:
        print('''
=== Personal Expense Tracker ===
1) Add Expense
2) View Expenses
3) Delete Expense
4) Update Expense
5) Plot Summary
6) total of month
7) Exit
        ''')
        choice = input('Choose an option: ')
        if choice == '1':
            add_expense()
        elif choice == '2':
            view_expenses()
        elif choice == '3':
            delete_expense()
        elif choice == '4':
            update_expense()
        elif choice == '5':
            plot_summary()
        elif choice == '6':
            monthly_category_summary()
        elif choice == '7':
            print('Goodbye!')
            break
        else:
            print('Invalid choice, try again.')


if __name__ == '__main__':
    main()